## Get data

In [1]:
from utils.mt5 import initialize_, get_currency_pair_data_

initialize_()

usdcad_df = get_currency_pair_data_('USDCAD')
usdcad_df.tail()

,open,high,low,close,tick_volume,spread
time,,,,,,
2024-09-08,1.35592,1.35701,1.35580,1.35627,1300,15
2024-09-09,1.35627,1.35763,1.35458,1.35635,20864,15
2024-09-10,1.35634,1.36147,1.35611,1.36096,22312,15
2024-09-11,1.36097,1.36221,1.35662,1.35778,27129,15
2024-09-12,1.35778,1.35846,1.35644,1.35678,4618,15


## Preprocess the data

In [2]:
import numpy as np

df = np.array(usdcad_df.copy()['close']).reshape(-1, 1)
df

array([[1.24685],
       [1.24769],
       [1.24424],
       ...,
       [1.36096],
       [1.35778],
       [1.35678]])

In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

data = scaler.fit_transform(df)
data

array([[0.23274365],
       [0.23730193],
       [0.21858042],
       ...,
       [0.8519644 ],
       [0.83470805],
       [0.82928153]])

In [4]:
def create_dataset(data, steps=1):
    X, y = [], []
    for i in range(len(data)-steps-1):
        X.append(data[i: i+steps, 0])
        y.append(data[i+steps, 0])
    return np.array(X), np.array(y)

In [5]:
steps = 60

split_idx = int(len(data) * 0.8)
train_data, test_data = data[:split_idx], data[split_idx:len(data)]

X_train, y_train = create_dataset(train_data, steps)
X_test, y_test = create_dataset(test_data, steps)

X_train.shape, X_test.shape

((815, 60), (158, 60))

In [6]:
X_train_3d = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_3d = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

X_train_3d.shape, X_test_3d.shape

((815, 60, 1), (158, 60, 1))

## Train a model

In [7]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Input, Dense, LSTM
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import MeanSquaredError

# model = Sequential()

# model.add(Input(shape=(steps, 1)))
# model.add(LSTM(100, return_sequences=True))
# model.add(LSTM(100, return_sequences=True))
# model.add(LSTM(100))
# model.add(Dense(1))

# model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=[MeanSquaredError()])
# model.fit(X_train_3d, y_train, batch_size=32, epochs=50, verbose=1);

In [8]:
import joblib 

# joblib.dump(model, 'models/lstm_USDCAD.pkl') 
model = joblib.load('models/lstm_USDCAD.pkl')  

## Testing 1,2,3...

In [9]:
train_predictions = scaler.inverse_transform(model.predict(X_train_3d))
test_predictions = scaler.inverse_transform(model.predict(X_test_3d))

test_predictions[:5]

26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


array([[1.3470572],
       [1.348495 ],
       [1.3458436],
       [1.3528553],
       [1.3529103]], dtype=float32)

In [10]:
train_actuals = scaler.inverse_transform(y_train.reshape(-1,1))
test_actuals = scaler.inverse_transform(y_test.reshape(-1,1))

test_actuals[:5]

array([[1.34914],
       [1.3465 ],
       [1.3537 ],
       [1.35405],
       [1.35455]])

In [11]:
from sklearn.metrics import mean_squared_error
import math 

train_error = math.sqrt(mean_squared_error(y_train, train_predictions))
test_error = math.sqrt(mean_squared_error(y_test, test_predictions))

train_error, test_error # |1> (0.7723888049452278, 0.4897638825407242), |2> (0.7706796807922176, 0.48687867349405445)

(0.7706796807922176, 0.48687867349405445)